In [0]:
from os.path import exists
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())
cuda_output = !ldconfig -p|grep cudart.so|sed -e 's/.*\.\([0-9]*\)\.\([0-9]*\)$/cu\1\2/'
accelerator = cuda_output[0] if exists('/dev/nvidia0') else 'cpu'

!pip install -q http://download.pytorch.org/whl/{accelerator}/torch-0.4.1-{platform}-linux_x86_64.whl torchvision
import torch

In [0]:
import torch
import numpy as np

# check if CUDA is available
train_on_gpu = torch.cuda.is_available()

if not train_on_gpu:
    print('CUDA is not available.  Training on CPU ...')
else:
    print('CUDA is available!  Training on GPU ...')
    
print(torch.__version__)

CUDA is available!  Training on GPU ...
0.4.1


In [0]:
# we need pillow version of 5.3.0
# we will uninstall the older version first
#!pip uninstall -y Pillow
# install the new one
#!pip install Pillow==5.3.0
# import the new one
import PIL
print(PIL.PILLOW_VERSION)
# this should print 5.3.0. If it doesn't, then restart your runtime:
# Menu > Runtime > Restart Runtime

5.3.0


In [0]:
# Imports here

# Imports here
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.utils.data.sampler import SubsetRandomSampler
from torch.autograd import Variable

from torchvision import datasets, transforms, models

from collections import OrderedDict

import matplotlib.pyplot as plt
%matplotlib inline

from PIL import Image

In [0]:
# we will download the required data files
!wget -cq https://github.com/udacity/pytorch_challenge/raw/master/cat_to_name.json
!wget -cq https://s3.amazonaws.com/content.udacity-data.com/courses/nd188/flower_data.zip
!rm -r flower_data || true
!unzip -qq flower_data.zip

rm: cannot remove 'flower_data': No such file or directory


In [0]:
!mkdir flower_data_original_test
!wget -cq https://www.dropbox.com/s/da6ye9genbsdzbq/flower_data_original_test.zip?dl=1
!unzip -qq flower_data_original_test.zip?dl=1 -d flower_data_original_test

In [0]:
!mkdir google_test_data
!wget -cq https://www.dropbox.com/s/3zmf1kq58o909rq/google_test_data.zip?dl=1;
!unzip -qq google_test_data.zip?dl=1 -d google_test_data

In [0]:
# Imports here
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data.sampler import SubsetRandomSampler
from torch.autograd import Variable

from torchvision import datasets, transforms, models

from collections import OrderedDict

import matplotlib.pyplot as plt
%matplotlib inline

import json

from PIL import Image

In [0]:
data_dir = 'flower_data'
train_dir = data_dir + '/train'
valid_dir = data_dir + '/valid'
test_dir = 'google_test_data/'

In [0]:

# TODO: Define your transforms for the training and validation sets
batch_size = 32
num_workers = 0

train_transform = transforms.Compose([transforms.RandomRotation(30),
                                      transforms.RandomResizedCrop(224),
                                      transforms.RandomHorizontalFlip(),
                                      transforms.ToTensor(),
                                      transforms.Normalize([0.485, 0.456, 0.406],
                                                           [0.229, 0.224, 0.225])
                                     ])

valid_transform = transforms.Compose([transforms.Resize(256),
                                      transforms.CenterCrop(224),
                                      transforms.ToTensor(),
                                      transforms.Normalize([0.485, 0.456, 0.406],
                                                           [0.229, 0.224, 0.225])
                                     ])

test_transform = transforms.Compose([transforms.Resize(256),
                                     transforms.CenterCrop(224),
                                     transforms.ToTensor(),
                                     transforms.Normalize([0.485, 0.456, 0.406],
                                                          [0.229, 0.224, 0.225])
                                    ])

# TODO: Load the datasets with ImageFolder
train_data = datasets.ImageFolder(train_dir, transform=train_transform)
valid_data = datasets.ImageFolder(valid_dir, transform=valid_transform)
test_data = datasets.ImageFolder(test_dir, transform = test_transform)

# TODO: Using the image datasets and the trainforms, define the dataloaders
train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, num_workers=num_workers, shuffle=True)
valid_loader = torch.utils.data.DataLoader(valid_data, batch_size=batch_size, num_workers=num_workers)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=batch_size, num_workers=num_workers)

In [0]:
with open('cat_to_name.json', 'r') as f:
    cat_to_name = json.load(f)
print(cat_to_name)

{'21': 'fire lily', '3': 'canterbury bells', '45': 'bolero deep blue', '1': 'pink primrose', '34': 'mexican aster', '27': 'prince of wales feathers', '7': 'moon orchid', '16': 'globe-flower', '25': 'grape hyacinth', '26': 'corn poppy', '79': 'toad lily', '39': 'siam tulip', '24': 'red ginger', '67': 'spring crocus', '35': 'alpine sea holly', '32': 'garden phlox', '10': 'globe thistle', '6': 'tiger lily', '93': 'ball moss', '33': 'love in the mist', '9': 'monkshood', '102': 'blackberry lily', '14': 'spear thistle', '19': 'balloon flower', '100': 'blanket flower', '13': 'king protea', '49': 'oxeye daisy', '15': 'yellow iris', '61': 'cautleya spicata', '31': 'carnation', '64': 'silverbush', '68': 'bearded iris', '63': 'black-eyed susan', '69': 'windflower', '62': 'japanese anemone', '20': 'giant white arum lily', '38': 'great masterwort', '4': 'sweet pea', '86': 'tree mallow', '101': 'trumpet creeper', '42': 'daffodil', '22': 'pincushion flower', '2': 'hard-leaved pocket orchid', '54': 's

In [0]:
# TODO: Build and train your network

# Using the resnet152 pre-trained network to get the features (The classifier portion is not needed)
model = models.densenet121(pretrained=True)

# Freeze all resnet parameters
for params in model.parameters():
    params.requires_grad = False

/usr/local/lib/python3.6/dist-packages/torchvision/models/densenet.py:212: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.
  nn.init.kaiming_normal(m.weight.data)


In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(model)

DenseNet(
  (features): Sequential(
    (conv0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (norm0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu0): ReLU(inplace)
    (pool0): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (denseblock1): _DenseBlock(
      (denselayer1): _DenseLayer(
        (norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplace)
        (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (norm2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu2): ReLU(inplace)
        (conv2): Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      )
      (denselayer2): _DenseLayer(
        (norm1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplac

In [0]:
# Define the classifier using nn.Sequential
classifier = nn.Sequential(OrderedDict([
    ('fc1', nn.Linear(1024, 102)),
    ('output', nn.LogSoftmax(dim=1))
]))
model.classifier = classifier
model.class_to_idx = train_data.class_to_idx
model.idx_to_class = {idx: cl for cl, idx in train_loader.dataset.class_to_idx.items()}

In [0]:
# Define the loss and optimizer
criterion = nn.NLLLoss()
optimizer = optim.Adam(model.classifier.parameters(), lr=0.001)

model.to(device)

DenseNet(
  (features): Sequential(
    (conv0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (norm0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu0): ReLU(inplace)
    (pool0): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (denseblock1): _DenseBlock(
      (denselayer1): _DenseLayer(
        (norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplace)
        (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (norm2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu2): ReLU(inplace)
        (conv2): Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      )
      (denselayer2): _DenseLayer(
        (norm1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplac

In [0]:
model_save_name = "densenet_121_aug2.pth"
path = F"/content/gdrive/My Drive/{model_save_name}" 

#Train the classifier using the features from the pre-trained network
epochs = 10

#Define a scheduler for the learning rate
scheduler = lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.1)

valid_loss_min = np.Inf # Track changes in validation loss

for e in range(1, epochs+1):
    # Keep track of training and validation loss
    train_loss = 0.0
    valid_loss = 0.0
    
    # train the model
    scheduler.step()
    model.train()
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        
        # Clear the gradients of all optimized variables
        optimizer.zero_grad()
        
        # Forward pass: compute predicted output by passing the inputs into the model
        outputs = model(images)
        # Calculate the batch loss
        loss = criterion(outputs, labels)
        # Backward pass: Compute gradient of the loss with respect to model parameters
        loss.backward()
        # Perform a single optimization step
        optimizer.step()
        
        # Update the training loss
        train_loss += loss.item()*images.size(0)
        
    #############################
    #### Validate the model #####
    #############################
    # put the model in evaluation mode
    model.eval()
    class_correct = list(0. for i in range(len(model.class_to_idx)))
    class_total = list(0. for i in range(102))
    
    for images, labels in valid_loader:
        # Move tensors to GPU if CUDA is available
        images, labels = images.to(device), labels.to(device)
        
        # Forward pass: compute predicted output by passing in inputs into the model
        outputs = model(images)
        # Calculate the batch loss
        loss = criterion(outputs, labels)
        
        # Update the validation loss
        valid_loss += loss.item()*images.size(0)
        # convert output probabilities to predicted class
        _, pred = torch.max(outputs, 1)    
        # compare predictions to true label
        correct_tensor = pred.eq(labels.data.view_as(pred))
        correct = np.squeeze(correct_tensor.numpy()) if not torch.cuda.is_available() else np.squeeze(correct_tensor.cpu().numpy())
        # calculate test accuracy for each object class
        for i in range(len(labels.data)):
            label = labels.data[i]
            class_correct[label] += correct[i].item()
            class_total[label] += 1
        
    # Calculate the average losses
    train_loss = train_loss/len(train_loader.dataset)
    valid_loss = valid_loss/len(valid_loader.dataset)
    
    print('Train Loss: {:.6f}\n'.format(train_loss))
    print('Valid Loss: {:.6f}\n'.format(valid_loss))
    
    # Print training/validation statistics
    print('Epoch :{} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}'.format(e, train_loss, valid_loss))
    
    # Save model if validation loss has decreased
    if valid_loss <= valid_loss_min:
        print('Validation loss decreased ({:.6f} ---------> {:.6f}). Saving model....'.format(valid_loss_min, valid_loss))
        checkpoint = {'state_dict': model.state_dict(),
                      'optimizer_state_dict': optimizer.state_dict,
                      'class_to_idx': model.class_to_idx,
                      'idx_to_class': model.idx_to_class}
        torch.save(checkpoint, path)
        valid_loss_min = valid_loss
        
    class_accuracies = {}
    for i in range(102):
        if class_total[i] > 0:
            class_accuracy = 100 * class_correct[i] / class_total[i]
            print('Validation Accuracy of %5s: %2d%% (%2d/%2d)' % (
                model.idx_to_class[i], class_accuracy,
                np.sum(class_correct[i]), np.sum(class_total[i])))
            # Keep the accuracy data for each class
            class_accuracies[i+1] = class_accuracy
        else:
            print('Validation Accuracy of %5s: N/A (no training examples)' % (model.idx_to_class[i]))

    total_acc = 100. * np.sum(class_correct) / np.sum(class_total)
    print('\nValidation Accuracy (Overall): %2d%% (%2d/%2d)' % (
        total_acc,
        np.sum(class_correct), np.sum(class_total)))

Train Loss: 2.869065

Valid Loss: 1.262948

Epoch :1 	Training Loss: 2.869065 	Validation Loss: 1.262948
Validation loss decreased (inf ---------> 1.262948). Saving model....
Validation Accuracy of     1: 87% ( 7/ 8)
Validation Accuracy of    10: 100% ( 4/ 4)
Validation Accuracy of   100:  0% ( 0/ 6)
Validation Accuracy of   101: 80% ( 4/ 5)
Validation Accuracy of   102: 66% ( 4/ 6)
Validation Accuracy of    11: 10% ( 1/10)
Validation Accuracy of    12: 100% ( 5/ 5)
Validation Accuracy of    13: 80% ( 4/ 5)
Validation Accuracy of    14: 100% ( 1/ 1)
Validation Accuracy of    15: 100% ( 7/ 7)
Validation Accuracy of    16: 50% ( 1/ 2)
Validation Accuracy of    17: 100% (16/16)
Validation Accuracy of    18: 81% ( 9/11)
Validation Accuracy of    19: 25% ( 1/ 4)
Validation Accuracy of     2: 100% ( 6/ 6)
Validation Accuracy of    20: 85% ( 6/ 7)
Validation Accuracy of    21: 100% ( 4/ 4)
Validation Accuracy of    22: 100% ( 8/ 8)
Validation Accuracy of    23: 100% (12/12)
Validation Accurac

In [0]:
# TODO: Write a function that loads a checkpoint and rebuilds the model
def load_checkpoint(filepath):
    model = models.densenet121(pretrained=False)
    
    classifier = nn.Sequential(OrderedDict([
    ('fc1', nn.Linear(1024, 102)),
    ('output', nn.LogSoftmax(dim=1))
]))
    
    model.classifier = classifier
    checkpoint = (torch.load(filepath))
    model.load_state_dict(checkpoint['state_dict'])
    model.optimizer = checkpoint['optimizer_state_dict']
    model.class_to_idx = checkpoint['class_to_idx']
    model.idx_to_class = checkpoint['idx_to_class']
    
    for param in model.parameters():
        param.requires_grad = False
        
    return model, model.class_to_idx, model.idx_to_class

model, model.class_to_idx, model.idx_to_class = load_checkpoint(path)

/usr/local/lib/python3.6/dist-packages/torchvision/models/densenet.py:212: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.
  nn.init.kaiming_normal(m.weight.data)


In [0]:
# track test loss
test_loss = 0.0
class_correct = list(0. for i in range(102))
class_total = list(0. for i in range(102))

# Move model to device
model.to(device)

model.eval()
# iterate over test data
for images, labels in test_loader:
    # move tensors to GPU if CUDA is available
    images, labels = images.to(device), labels.to(device)
    # forward pass: compute predicted outputs by passing inputs to the model
    output = model(images)
    # calculate the batch loss
    loss = criterion(output, labels)
    # update test loss 
    test_loss += loss.item()*images.size(0)
    # convert output probabilities to predicted class
    _, pred = torch.max(output, 1)    
    # compare predictions to true label
    correct_tensor = pred.eq(labels.data.view_as(pred))
    correct = np.squeeze(correct_tensor.numpy()) if not torch.cuda.is_available() else np.squeeze(correct_tensor.cpu().numpy())
    # calculate test accuracy for each object class
    for i in range(len(labels.data)):
        label = labels.data[i]
        class_correct[label] += correct[i].item()
        class_total[label] += 1

# average test loss
test_loss = test_loss/len(test_loader.dataset)
print('Test Loss: {:.6f}\n'.format(test_loss))

class_accuracies = {}
for i in range(102):
    if class_total[i] > 0:
        class_accuracy = 100 * class_correct[i] / class_total[i]
        print('Test Accuracy of %5s: %2d%% (%2d/%2d)' % (
            model.idx_to_class[i], class_accuracy,
            np.sum(class_correct[i]), np.sum(class_total[i])))
        # Keep the accuracy data for each class
        class_accuracies[i+1] = class_accuracy
    else:
        print('Test Accuracy of %5s: N/A (no training examples)' % (model.idx_to_class[i]))

total_acc = 100. * np.sum(class_correct) / np.sum(class_total)
print('\nTest Accuracy (Overall): %2d%% (%2d/%2d)' % (
    total_acc,
    np.sum(class_correct), np.sum(class_total)))

Test Loss: 0.420286

Test Accuracy of     1: 80% ( 4/ 5)
Test Accuracy of    10: 100% ( 3/ 3)
Test Accuracy of   100: 100% ( 8/ 8)
Test Accuracy of   101: 100% ( 4/ 4)
Test Accuracy of   102: 100% ( 6/ 6)
Test Accuracy of    11: 88% ( 8/ 9)
Test Accuracy of    12: 88% ( 8/ 9)
Test Accuracy of    13: 83% ( 5/ 6)
Test Accuracy of    14: 100% ( 3/ 3)
Test Accuracy of    15: 100% ( 4/ 4)
Test Accuracy of    16: 100% ( 3/ 3)
Test Accuracy of    17: 100% ( 9/ 9)
Test Accuracy of    18: 100% ( 6/ 6)
Test Accuracy of    19: 100% ( 7/ 7)
Test Accuracy of     2: 100% ( 5/ 5)
Test Accuracy of    20: 66% ( 2/ 3)
Test Accuracy of    21: 50% ( 1/ 2)
Test Accuracy of    22: 100% ( 4/ 4)
Test Accuracy of    23: 100% ( 7/ 7)
Test Accuracy of    24: 100% ( 2/ 2)
Test Accuracy of    25: 100% ( 5/ 5)
Test Accuracy of    26: 80% ( 4/ 5)
Test Accuracy of    27: 100% ( 3/ 3)
Test Accuracy of    28: 100% ( 6/ 6)
Test Accuracy of    29: 88% ( 8/ 9)
Test Accuracy of     3: 100% ( 2/ 2)
Test Accuracy of    30: 1